## BlueSky Airlines Case (Revisited)

Python packages for simulation:

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Given the demand for FF, $D_{F}$, and the demand for SS, $D_S$, the revenue with a SS booking limit equal to 120 is

$$
R = 350 * \min(D_S, 120) + 850 * \min(D_F, 200 - \min(D_S, 120))
$$


### Creating our baseline simulation

In [2]:
# set the number of simulations, numRep
numRep = 50000

# define two fare types, SS and FF
fareType = ['SS', 'FF']

# define mean and standard deviations dictionaries for the two fare types: 130 and 45 for SS, 55 and 20 for FF
mean = {'SS': 130, 'FF': 55}
stdev = {'SS': 45, 'FF': 20}

# define prices for the two fare types: 350 for SS, 850 for FF
price = {'SS': 350, 'FF': 850}

# define the booking limit for SS, 120
bookLimitSS = 120

# define the plane capacity, 200
planeCapacity = 200

# set the seed
np.random.seed(18)

In [3]:
revenues = np.zeros(numRep)
prob_one_empty = 0
prob_low_revenue = 0

for rep in range(numRep):
    #generate a normal random variable called DS with mean mean[SS] and standard deviation stdev[SS]
    DS = np.random.normal(mean['SS'], stdev['SS'])
    DS = max(0, DS)  # Ensure DS is non-negative

    #round DS to the nearest integer
    DS = round(DS)

    #generate a normal random variable called DF with mean mean[FF] and standard deviation stdev[FF]
    DF = np.random.normal(mean['FF'], stdev['FF'])
    DF = max(0, DF)

    #round DF to the nearest integer
    DF = round(DF)

    XS = min(DS, 120)
    XF = min(DF, 200 - XS)

    rev = 350 * XS + 850 * XF
    revenues[rep] = rev

    if XS + XF < planeCapacity:
        prob_one_empty += 1

    if rev < 75000:
        prob_low_revenue += 1

prob_one_empty /= numRep
prob_low_revenue /= numRep

In [4]:
#print the average revenue with zero digits
print("The average revenue is", round(np.mean(revenues), 0))

#compute a 95% confidence interval for the average revenue
CI_up   = np.mean(revenues) + 1.96 * np.std(revenues) / np.sqrt(numRep)
CI_down = np.mean(revenues) - 1.96 * np.std(revenues) / np.sqrt(numRep)

#print the confidence interval with zero digits
print("The 95% confidence interval is", round(CI_down, 0), "to", round(CI_up, 0))

#print the probability that there is at least one empty seat
print("The probability that there is at least one empty seat is", prob_one_empty)

#print the probability that the revenue is less than 75,000
print("The probability that the revenue is less than 75,000 is", prob_low_revenue)

The average revenue is 83419.0
The 95% confidence interval is 83264.0 to 83575.0
The probability that there is at least one empty seat is 0.92596
The probability that the revenue is less than 75,000 is 0.3033


### Overbooking (alternate model)

A simpler model of overbooking than the one presented in the case is as follows:
 - Round Demand so that it is a non-negative integer. 
 - Each person independently "flips a coin" and shows up with probability $p_s$ or $p_F$ depending on their fare class.

Use this modelto simulate expected revenue.

In [5]:
# define virtual capacity as 220
virtualCapacity = 220

# set the booking limit to 160
bookLimit = 160

# define a dictionary for the overbook cost for SS and FF to 450 and 1000, respectively
overbookCost = {'SS': 450, 'FF': 1000}

# define a list of revenues for each replication in the numSim simulations, and initialize it as a list of zeros
revenue = np.zeros(numRep)

#probabilities
ps = .92
pf = .95

In [6]:
for n in range(numRep):
    # generate a normal random variable for the SS demand using the mean and standard deviation defined above
    demandSS = np.random.normal(mean['SS'], stdev['SS'])
    
    # generate a normal random variable for the FF demand using the mean and standard deviation defined above
    demandFF = np.random.normal(mean['FF'], stdev['FF'])
    
    # calculate the number of tickets sold and the number of arrivals of each type of passengers
    X_S = min(demandSS, bookLimit)
    X_F = min(demandFF, virtualCapacity - X_S)